# Get a list of Targets based on a pathway
#### Create a POST request to the RaMP API

In [ ]:
import requests
import pandas as pd
import json

# see https://rampdb.nih.gov/api for details

ramp_url = "https://rampdb.nih.gov/api/analytes-from-pathways"

post_body = {
    "analyte_type": "gene",
    "names_or_ids": "names",
    "max_pathway_size": 500,
    "pathway": ["Sleep regulation"]
}

#### do the POST request

In [ ]:
response = requests.post(ramp_url, data=post_body)
response_json = json.loads(response.text)["data"]

#### Parse the result

In [ ]:
target_uniprots = []
for entry in response_json:
    ids = entry['sourceAnalyteIDs'].split(',')
    for id in ids:
        if id.startswith('uniprot:'):
            uniprot = id.split(':')[1]
            target_uniprots.append(uniprot)
            
target_uniprots = list(set(target_uniprots))

print(pd.DataFrame(target_uniprots))

# Get gene-disease associations and drug activities for those targets
#### Create a POST request to the Pharos API

In [ ]:
# see https://pharos.nih.gov/api for details

pharos_url = "https://pharos-api.ncats.io/graphql"

post_body = {
  "query": """query targetDetails($targets: [String]){
  targets(targets: $targets) {
    count
    targets(top: 100000) {
      name
      uniprot
      drugs:ligands(isdrug: true) {
        name
        smiles
        synonyms {
          value
          name
        }
      }
      ligands(isdrug: false) {
        name
        smiles
        synonyms {
          value
          name
        }
      }
      diseases {
        mondoID
        name
        mondoEquivalents {
          id
          name
        }
      }
    }
  }
}""",
  "variables": {"targets": target_uniprots},
  "operationName": "targetDetails"
}

#### do the POST request

In [ ]:
response = requests.post(pharos_url, json=post_body)
response_json = json.loads(response.text)["data"]
target_list = response_json["targets"]["targets"]

#### Parse the disease results

In [ ]:
disease_objects = [disease for target in target_list for disease in target["diseases"]]
disease_names = list(set(disease["name"] for disease in disease_objects))

print(pd.DataFrame(disease_objects))

gard_aliases = [alias["id"] for disease in disease_objects if disease["mondoEquivalents"] is not None for alias in disease["mondoEquivalents"] if alias["id"].startswith("GARD:")]
gard_aliases = list(set(gard_aliases))

print(gard_aliases)

#### Parse the drug results

In [ ]:
drug_objects = [drug for target in target_list for drug in target["ligands"]]
drug_objects.extend([drug for target in target_list for drug in target["drugs"]])

drug_names = list(set(drug["name"] for drug in drug_objects))

print(pd.DataFrame(drug_objects))

unii_aliases = [synonym["value"] for drug in drug_objects for synonym in drug["synonyms"] if synonym["name"] == "unii"]
unii_aliases = list(set(unii_aliases))
print(unii_aliases)

## Get Disease Details
#### Create POST request to the RDAS API

In [ ]:
# see https://rdas.ncats.nih.gov/api/diseases for details

rdas_url = "https://rdas.ncats.nih.gov/api/diseases"

post_body = {
    "query": """query diseaseQuery($where: GARDWhere) {
  gards(where: $where) {
    GardId
    GardName
    MeSH
    Synonyms
    COUNT_ARTICLES
    COUNT_TRIALS
    ClassificationLevel
  }
}""", 

  "variables": {"where": {"GardId_IN": gard_aliases}},
  "operationName": "diseaseQuery"
}

#### do the POST request

In [ ]:
response = requests.post(rdas_url, json=post_body)
response_json = json.loads(response.text)["data"]

#### parse the results

In [ ]:
gard_data = [{
    "GardId": obj["GardId"],
    "GardName": obj["GardName"],
    "COUNT_ARTICLES": obj["COUNT_ARTICLES"],
    "COUNT_TRIALS": obj["COUNT_TRIALS"],
    "ClassificationLevel": obj["ClassificationLevel"]
} for obj in response_json["gards"]]
df = pd.DataFrame(gard_data)
print(df)

## Get Drug Details
#### Create GET request strings

In [ ]:
drug_requests = [f"https://drugs.ncats.io/api/v1/substances({unii})?view=full" for unii in unii_aliases]

drug_objects = []
for req in drug_requests:
    
    drug_json = json.loads(requests.get(req).text)
    
    if "_name" in drug_json:
        drug_obj = {
            "name": drug_json["_name"], 
            "approval_ID": drug_json["approvalID"],
            "substanceClass": drug_json["substanceClass"],
            "status": drug_json["status"]
        }
        if "structure" in drug_json:
            drug_obj["formula"] = drug_json["structure"]["formula"]
            drug_obj["mwt"] = drug_json["structure"]["mwt"]
        
        drug_objects.append(drug_obj)
    else:
        print(drug_json)

df = pd.DataFrame(drug_objects)
print(df)